In [44]:
try:
    spark.stop()
except:
    pass


قم بإنشاء جلسة Spark لتحميل ومعالجة البيانات:

In [45]:
from pyspark.sql import SparkSession
import os

os.environ["PYSPARK_PYTHON"] = r"c:\Users\162833\AppData\Local\miniconda3\envs\food_project\python.exe"

spark = SparkSession.builder \
    .appName("FoodDeliveryAnalytics") \
    .config("spark.driver.bindAddress", "127.0.0.1") \
    .config("spark.driver.host", "127.0.0.1") \
    .getOrCreate()


استخدم spark.read لتحميل البيانات من ملفات CSV أو JSON:

In [46]:
from pyspark.sql import SparkSession

# إنشاء جلسة Spark
spark = SparkSession.builder.appName("FoodDeliveryAnalytics").getOrCreate()

# تعريف المسار الأساسي
data_path = "C:/me/food_delivery_analytics/data/data1"

# تحميل ملف CSV (zomato)
df_zomato = spark.read.csv(f"{data_path}/zomato.csv", header=True, inferSchema=True)

# تحميل ملفات JSON
df_file1 = spark.read.json(f"{data_path}/file1.json")
df_file2 = spark.read.json(f"{data_path}/file2.json")
df_file3 = spark.read.json(f"{data_path}/file3.json")
df_file4 = spark.read.json(f"{data_path}/file4.json")
df_file5 = spark.read.json(f"{data_path}/file5.json")

# تحميل ملف Excel (Country-Code.xlsx)
# نستخدم Pandas + تحويله إلى Spark DataFrame
import pandas as pd

df_country_code_pd = pd.read_excel(f"{data_path}/Country-Code.xlsx")
df_country_code_pd = df_country_code_pd.astype(str)
df_country_code = spark.createDataFrame(df_country_code_pd)

# تأكيد التحميل
df_zomato.show(3)
df_file1.show(3)
df_country_code.show(3)


+-------------+--------------------+------------+----------------+--------------------+--------------------+--------------------+----------+---------+--------------------+--------------------+----------------+-----------------+-------------------+-----------------+--------------------+-----------+----------------+------------+-----------+-----+
|Restaurant ID|     Restaurant Name|Country Code|            City|             Address|            Locality|    Locality Verbose| Longitude| Latitude|            Cuisines|Average Cost for two|        Currency|Has Table booking|Has Online delivery|Is delivering now|Switch to order menu|Price range|Aggregate rating|Rating color|Rating text|Votes|
+-------------+--------------------+------------+----------------+--------------------+--------------------+--------------------+----------+---------+--------------------+--------------------+----------------+-----------------+-------------------+-----------------+--------------------+-----------+--------

هناخد دول هنا ونشتغل عليهم علشان نظبط الداتا

In [47]:
# تحميل ملفات JSON
df_file1 = spark.read.json(f"{data_path}/file1.json")
df_file2 = spark.read.json(f"{data_path}/file2.json")
df_file3 = spark.read.json(f"{data_path}/file3.json")
df_file4 = spark.read.json(f"{data_path}/file4.json")
df_file5 = spark.read.json(f"{data_path}/file5.json")

df_file1.show(3)

+----+-------+--------------------+-------------+-------------+-------------+------+
|code|message|         restaurants|results_found|results_shown|results_start|status|
+----+-------+--------------------+-------------+-------------+-------------+------+
|NULL|   NULL|[{{{308322}, b90e...|        17151|           20|            1|  NULL|
|NULL|   NULL|[{{{18017612}, b9...|         4748|           20|            1|  NULL|
|NULL|   NULL|[{{{18313566}, b9...|        13786|           20|            1|  NULL|
+----+-------+--------------------+-------------+-------------+-------------+------+
only showing top 3 rows



انا هنا اشتغلت علي الفايل بتاع json وخدت من العمود اللى اسمه المطعم وفردت الداتا اللى فيه فى جدول 

In [48]:
from pyspark.sql.functions import explode, col

# قائمة تحتوي على جميع DataFrames الخاصة بملفات JSON
json_files = [df_file1, df_file2, df_file3, df_file4, df_file5]

# الأعمدة المطلوبة بعد التفكيك
columns = [
    col("restaurant.id").alias("Restaurant ID"),
    col("restaurant.name").alias("Restaurant Name"),
    col("restaurant.location.country_id").alias("Country Code"),
    col("restaurant.location.city").alias("City"),
    col("restaurant.location.address").alias("Address"),
    col("restaurant.location.locality").alias("Locality"),
    col("restaurant.location.locality_verbose").alias("Locality Verbose"),
    col("restaurant.location.longitude").alias("Longitude"),
    col("restaurant.location.latitude").alias("Latitude"),
    col("restaurant.cuisines").alias("Cuisines"),
    col("restaurant.average_cost_for_two").alias("Average Cost for two"),
    col("restaurant.currency").alias("Currency"),
    col("restaurant.has_table_booking").alias("Has Table Booking"),
    col("restaurant.has_online_delivery").alias("Has Online delivery"),
    col("restaurant.is_delivering_now").alias("Is delivering now"),
    col("restaurant.switch_to_order_menu").alias("Switch to order menu"),
    col("restaurant.price_range").alias("Price range"),
    col("restaurant.user_rating.aggregate_rating").alias("Aggregate rating"),
    col("restaurant.user_rating.rating_color").alias("Rating color"),
    col("restaurant.user_rating.rating_text").alias("Rating text"),
    col("restaurant.user_rating.votes").alias("Votes")
]

# تفكيك جميع الملفات ودمجها في DataFrame واحد
df_combined = None  # متغير لحفظ النتيجة النهائية

for i, df in enumerate(json_files):
    # تفكيك عمود المطاعم
    df_exploded = df.withColumn("restaurant", explode(col("restaurants.restaurant")))
    
    # استخراج الأعمدة المطلوبة
    df_flat = df_exploded.select(*columns)
    
    # دمج البيانات مع DataFrame النهائي
    if df_combined is None:
        df_combined = df_flat  # أول ملف يتم تعيينه مباشرة
    else:
        df_combined = df_combined.unionByName(df_flat)  # دمج الملفات الأخرى

# عرض البيانات المدمجة
df_combined.show(10, truncate=False)


+-------------+--------------------------------+------------+---------+-------------------------------------------------------------------------+----------------------------------------+-------------------------------------------------+-------------+-------------+---------------------------------------------------------+--------------------+--------+-----------------+-------------------+-----------------+--------------------+-----------+----------------+------------+-----------+-----+
|Restaurant ID|Restaurant Name                 |Country Code|City     |Address                                                                  |Locality                                |Locality Verbose                                 |Longitude    |Latitude     |Cuisines                                                 |Average Cost for two|Currency|Has Table Booking|Has Online delivery|Is delivering now|Switch to order menu|Price range|Aggregate rating|Rating color|Rating text|Votes|
+-------------+-----

In [49]:
from pyspark.sql.functions import lit

# تأكد أن الأعمدة بنفس الترتيب في الداتا فريمين
columns = [
    "Restaurant ID", "Restaurant Name", "Country Code", "City", "Address", "Locality", "Locality Verbose",
    "Longitude", "Latitude", "Cuisines", "Average Cost for two", "Currency", "Has Table Booking",
    "Has Online delivery", "Is delivering now", "Switch to order menu", "Price range", "Aggregate rating",
    "Rating color", "Rating text", "Votes"
]

# إعادة ترتيب الأعمدة في الداتا فريم الأول
df_flat = df_flat.select(*columns)
# إعادة ترتيب الأعمدة في الداتا فريم الثاني
df_zomato = df_zomato.select(*columns)

# دمج الداتا فريمين
df_merged = df_flat.unionByName(df_zomato)


In [50]:
df_merged.show(10, truncate=False)


+-------------+------------------------------------+------------+-------+----------------------------------------------------+-----------------------+--------------------------------+--------------+-------------+---------------------------------+--------------------+--------+-----------------+-------------------+-----------------+--------------------+-----------+----------------+------------+-----------+-----+
|Restaurant ID|Restaurant Name                     |Country Code|City   |Address                                             |Locality               |Locality Verbose                |Longitude     |Latitude     |Cuisines                         |Average Cost for two|Currency|Has Table Booking|Has Online delivery|Is delivering now|Switch to order menu|Price range|Aggregate rating|Rating color|Rating text|Votes|
+-------------+------------------------------------+------------+-------+----------------------------------------------------+-----------------------+----------------------

لعرض انواع الاعمدة

In [51]:
df_merged.printSchema()


root
 |-- Restaurant ID: string (nullable = true)
 |-- Restaurant Name: string (nullable = true)
 |-- Country Code: string (nullable = true)
 |-- City: string (nullable = true)
 |-- Address: string (nullable = true)
 |-- Locality: string (nullable = true)
 |-- Locality Verbose: string (nullable = true)
 |-- Longitude: string (nullable = true)
 |-- Latitude: string (nullable = true)
 |-- Cuisines: string (nullable = true)
 |-- Average Cost for two: string (nullable = true)
 |-- Currency: string (nullable = true)
 |-- Has Table Booking: string (nullable = true)
 |-- Has Online delivery: string (nullable = true)
 |-- Is delivering now: string (nullable = true)
 |-- Switch to order menu: string (nullable = true)
 |-- Price range: string (nullable = true)
 |-- Aggregate rating: string (nullable = true)
 |-- Rating color: string (nullable = true)
 |-- Rating text: string (nullable = true)
 |-- Votes: string (nullable = true)



In [52]:
from pyspark.sql.functions import col

df_merged = df_merged \
    .withColumn("Restaurant ID", col("Restaurant ID").cast("int")) \
    .withColumn("Country Code", col("Country Code").cast("int")) \
    .withColumn("Longitude", col("Longitude").cast("double")) \
    .withColumn("Latitude", col("Latitude").cast("double")) \
    .withColumn("Average Cost for two", col("Average Cost for two").cast("int")) \
    .withColumn("Price range", col("Price range").cast("int")) \
    .withColumn("Aggregate rating", col("Aggregate rating").cast("double")) \
    .withColumn("Votes", col("Votes").cast("int"))
df_merged.printSchema()  # تأكيد التغييرات في الأنواع

root
 |-- Restaurant ID: integer (nullable = true)
 |-- Restaurant Name: string (nullable = true)
 |-- Country Code: integer (nullable = true)
 |-- City: string (nullable = true)
 |-- Address: string (nullable = true)
 |-- Locality: string (nullable = true)
 |-- Locality Verbose: string (nullable = true)
 |-- Longitude: double (nullable = true)
 |-- Latitude: double (nullable = true)
 |-- Cuisines: string (nullable = true)
 |-- Average Cost for two: integer (nullable = true)
 |-- Currency: string (nullable = true)
 |-- Has Table Booking: string (nullable = true)
 |-- Has Online delivery: string (nullable = true)
 |-- Is delivering now: string (nullable = true)
 |-- Switch to order menu: string (nullable = true)
 |-- Price range: integer (nullable = true)
 |-- Aggregate rating: double (nullable = true)
 |-- Rating color: string (nullable = true)
 |-- Rating text: string (nullable = true)
 |-- Votes: integer (nullable = true)



In [53]:
df_final = df_merged
df_final.show()


+-------------+--------------------+------------+-------+--------------------+--------------------+--------------------+-----------+----------+--------------------+--------------------+--------+-----------------+-------------------+-----------------+--------------------+-----------+----------------+------------+-----------+-----+
|Restaurant ID|     Restaurant Name|Country Code|   City|             Address|            Locality|    Locality Verbose|  Longitude|  Latitude|            Cuisines|Average Cost for two|Currency|Has Table Booking|Has Online delivery|Is delivering now|Switch to order menu|Price range|Aggregate rating|Rating color|Rating text|Votes|
+-------------+--------------------+------------+-------+--------------------+--------------------+--------------------+-----------+----------+--------------------+--------------------+--------+-----------------+-------------------+-----------------+--------------------+-----------+----------------+------------+-----------+-----+
|   

In [56]:
# ----------------------------------------
# إعداد بيئة العمل وإنشاء جلسة Spark
# ----------------------------------------
from pyspark.sql import SparkSession
from pyspark.sql.functions import explode, col
import pandas as pd
import os

# تحديد مسار Python الخاص بـ PySpark (مناسب لبيئة conda الخاصة بك)
os.environ["PYSPARK_PYTHON"] = r"c:\Users\162833\AppData\Local\miniconda3\envs\food_project\python.exe"

# إنشاء جلسة Spark
spark = SparkSession.builder \
    .appName("FoodDeliveryAnalytics") \
    .config("spark.driver.bindAddress", "127.0.0.1") \
    .config("spark.driver.host", "127.0.0.1") \
    .getOrCreate()

# ----------------------------------------
# تحميل البيانات من الملفات
# ----------------------------------------

# المسار الرئيسي للبيانات
data_path = "C:/me/food_delivery_analytics/data/data1"

# تحميل ملف CSV (بيانات Zomato)
df_zomato = spark.read.csv(f"{data_path}/zomato.csv", header=True, inferSchema=True)

# تحميل جميع ملفات JSON (file1 إلى file5)
json_files = [f"{data_path}/file{i}.json" for i in range(1, 6)]
df_json_combined = None

for file in json_files:
    df_json = spark.read.json(file)
    # استخراج وتفكيك بيانات المطاعم
    df_exploded = df_json.withColumn("restaurant", explode(col("restaurants.restaurant")))
    
    # تسطيح البيانات المطلوبة
    df_flat = df_exploded.select(
        col("restaurant.id").alias("Restaurant ID"),
        col("restaurant.name").alias("Restaurant Name"),
        col("restaurant.location.country_id").alias("Country Code"),
        col("restaurant.location.city").alias("City"),
        col("restaurant.location.address").alias("Address"),
        col("restaurant.location.locality").alias("Locality"),
        col("restaurant.location.locality_verbose").alias("Locality Verbose"),
        col("restaurant.location.longitude").alias("Longitude"),
        col("restaurant.location.latitude").alias("Latitude"),
        col("restaurant.cuisines").alias("Cuisines"),
        col("restaurant.average_cost_for_two").alias("Average Cost for two"),
        col("restaurant.currency").alias("Currency"),
        col("restaurant.has_table_booking").alias("Has Table Booking"),
        col("restaurant.has_online_delivery").alias("Has Online delivery"),
        col("restaurant.is_delivering_now").alias("Is delivering now"),
        col("restaurant.switch_to_order_menu").alias("Switch to order menu"),
        col("restaurant.price_range").alias("Price range"),
        col("restaurant.user_rating.aggregate_rating").alias("Aggregate rating"),
        col("restaurant.user_rating.rating_color").alias("Rating color"),
        col("restaurant.user_rating.rating_text").alias("Rating text"),
        col("restaurant.user_rating.votes").alias("Votes"),
    )

    # دمج كل ملفات JSON في DataFrame واحد
    df_json_combined = df_flat if df_json_combined is None else df_json_combined.unionByName(df_flat)

# ----------------------------------------
# تحميل ملف Excel الخاص بـ Country Codes
# ----------------------------------------
df_country_code_pd = pd.read_excel(f"{data_path}/Country-Code.xlsx").astype(str)
df_country_code = spark.createDataFrame(df_country_code_pd)

# ----------------------------------------
# توحيد الأعمدة بين zomato وملفات JSON
# ----------------------------------------

columns = [
    "Restaurant ID", "Restaurant Name", "Country Code", "City", "Address", "Locality", "Locality Verbose",
    "Longitude", "Latitude", "Cuisines", "Average Cost for two", "Currency", "Has Table Booking",
    "Has Online delivery", "Is delivering now", "Switch to order menu", "Price range", "Aggregate rating",
    "Rating color", "Rating text", "Votes"
]

# التأكد من ترتيب الأعمدة
df_zomato = df_zomato.select(*columns)
df_json_combined = df_json_combined.select(*columns)

# دمج بيانات JSON مع بيانات CSV
df_merged = df_json_combined.unionByName(df_zomato)

# ----------------------------------------
# تحويل أنواع البيانات حسب الاستخدام
# ----------------------------------------

df_merged = df_merged \
    .withColumn("Restaurant ID", col("Restaurant ID").cast("int")) \
    .withColumn("Country Code", col("Country Code").cast("int")) \
    .withColumn("Longitude", col("Longitude").cast("double")) \
    .withColumn("Latitude", col("Latitude").cast("double")) \
    .withColumn("Average Cost for two", col("Average Cost for two").cast("int")) \
    .withColumn("Price range", col("Price range").cast("int")) \
    .withColumn("Aggregate rating", col("Aggregate rating").cast("double")) \
    .withColumn("Votes", col("Votes").cast("int"))

# ----------------------------------------
# عرض النتائج النهائية
# ----------------------------------------

df_merged.printSchema()
df_merged.show(10, truncate=False)


root
 |-- Restaurant ID: integer (nullable = true)
 |-- Restaurant Name: string (nullable = true)
 |-- Country Code: integer (nullable = true)
 |-- City: string (nullable = true)
 |-- Address: string (nullable = true)
 |-- Locality: string (nullable = true)
 |-- Locality Verbose: string (nullable = true)
 |-- Longitude: double (nullable = true)
 |-- Latitude: double (nullable = true)
 |-- Cuisines: string (nullable = true)
 |-- Average Cost for two: integer (nullable = true)
 |-- Currency: string (nullable = true)
 |-- Has Table Booking: string (nullable = true)
 |-- Has Online delivery: string (nullable = true)
 |-- Is delivering now: string (nullable = true)
 |-- Switch to order menu: string (nullable = true)
 |-- Price range: integer (nullable = true)
 |-- Aggregate rating: double (nullable = true)
 |-- Rating color: string (nullable = true)
 |-- Rating text: string (nullable = true)
 |-- Votes: integer (nullable = true)

+-------------+--------------------------------+------------+